# Import the necessary libraries

In [1]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import math
from loop_analysis import *
from sklearn.linear_model import LinearRegression

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [42]:
filefactor = str(file_name).split("_")[0] + str('.') + str(file_name).split("_")[1]
float(filefactor)

1.0

## Plot averaged loops

In [13]:
pi = 45
direction = 'y'
file_name = 'ew45m2.h5'
signal_type = 'woll1_diff'
############################################# Load file data ###################################################################
#folder_name = r'C:\Users\migels\Miguel\PhD\MOKE\Measures\W1\Measurement2\Pi'+str(pi)+'\B'+str(direction)
folder_name = r'C:\Users\migels\Miguel\PhD\MOKE\Measures\NdCo520nm\B'+str(direction)
filepath = os.path.join(folder_name, file_name)
data = get_averaged_loop(filepath,1000)
# Correct the difference and sum of signals from the detectors
data['woll1_diff'] = data['woll1_det1'] - data['woll1_det2']
data['woll1_sum'] = data['woll1_det1'] + data['woll1_det2']
## Title construction
# What we are measuring: rotation or ellipticity
if file_name[0] == 'e': 
    mtype = 'ellipticity'
else:
    mtype = 'rotation'
# How we are using our prism
if file_name[2] == str(4):
    if file_name[4] == 'm':
        wollo = 'wollaston at 45 manually'        
    else:
        wollo = 'wollaston at 45 based on detectors'
else:
    wollo = 'wollaston aligned with s and p'   
  
incidence_angle = (180 - 2*pi)/2
full_title = 'NdCo5 ' + str(mtype) + ' with ' + str(wollo) + ' at ' + str(incidence_angle)+ ' deg'  
################################################################################################################################
############################################### Here we choose what signal to use ##############################################
cutoff1 = -30
cutoff2 = 30
################################################################################################################################
######################################### Extract the data and manipulate ######################################################
# Get magnetic field on sample reference frame
y = data[signal_type]
y,a = straighten_loop(data,cutoff1,cutoff2,direction,signal_type)
bxt = np.array(data['fields_X'])
byt = np.array(data['fields_Y'])
bzt = np.array(data['fields_Z'])
Bxsample = []
for i in range(len(bxt)):
    aux = math.sqrt(bxt[i]**2 + bzt[i]**2)
    if bxt[i] < 0:
        Bxsample.append(-aux)
    else:
        Bxsample.append(aux)

# Create figure
%matplotlib qt
fig=plt.figure(figsize=(9, 5), dpi= 200, facecolor='w', edgecolor='k')
plt.rcParams.update({'font.size': 16})
if direction=='x':
    plt.plot(Bxsample, y, linewidth = 3)
else:
    plt.plot(byt, y, linewidth = 3)
plt.grid()
plt.xlabel('B' + str(direction) + ' [mT]')
if signal_type == 'woll1_diff': tipo = ' difference '
if signal_type == 'woll1_sum': tipo = ' sum '
if signal_type == 'woll1_det1': tipo = ' detector 1 '
if signal_type == 'woll1_det2': tipo = ' detector 2 '    
plt.ylabel('MOKE' + tipo + 'signal')
#plt.xlim(-9,9)
plt.title(full_title)
plt.tight_layout()
saving_name = os.path.splitext(filepath)[0]+'_'+signal_type+'.png'
#plt.savefig(saving_name)

## Demagnetization

In [83]:
file_name = '1_8.h5'
mtype = 'Samp'
signal_type = 'woll1_diff'
direction = 'x'
OAMPS = np.array([3,1.3,1.08,1.03,1.01,0.99])
############################################# Load file data ###################################################################
folder_name = r'C:\Users\user\Desktop\Miguel\Demag\\' + str(mtype)
filepath = os.path.join(folder_name, file_name)
data = get_averaged_loop(filepath,1000)
# Correct the difference and sum of signals from the detectors
data['woll1_diff'] = data['woll1_det1'] - data['woll1_det2']
data['woll1_sum'] = data['woll1_det1'] + data['woll1_det2']

################################################################################################################################
############################################### Here we choose what signal to use ##############################################
cutoff1 = -2
cutoff2 = 6
################################################################################################################################
######################################### Extract the data and manipulate ######################################################
# Get magnetic field on sample reference frame
y = data[signal_type]
y,a = straighten_loop(data,cutoff1,cutoff2,direction,signal_type)


#mn = np.min(y)
#mx = np.max(y)
#dif = abs(mx - mn)
#if mn < 0:
#    y = y + abs(mn) - dif/2
#else:
#    y = y - abs(mn) - dif/2


bxt = np.array(data['fields_X'])
byt = np.array(data['fields_Y'])
bzt = np.array(data['fields_Z'])
Bxsample = []
for i in range(len(bxt)):
    aux = math.sqrt(bxt[i]**2 + bzt[i]**2)
    if bxt[i] < 0:
        Bxsample.append(-aux)
    else:
        Bxsample.append(aux)

# Create figure
%matplotlib qt
fig, ax = plt.subplots(figsize=(9, 5), dpi= 200, facecolor='w', edgecolor='k')
plt.rcParams.update({'font.size': 16})
if direction=='x':
    plt.plot(Bxsample, y, linewidth = 3)
else:
    plt.plot(byt, y, linewidth = 3)
plt.grid()

if mtype == 'MultipleAmalio': ftitle = 'Varying the initial amplitudes in unison by a factor ' + str(file_name).split(".")[0]
if mtype == 'FreqVar': ftitle = 'Duration of signal at ' + str(file_name).split(".")[0] + ' s'
if mtype == 'Samp': ftitle = 'Second amplitude of sinusoid at ' + str(file_name).split(".")[0]

number = str(file_name).split(".")[0]
filefactor = str(number).split("_")[0] + str('.') + str(number).split("_")[1]
filefactor = float(filefactor)

if mtype == 'MultipleAmalio': OAMPS = OAMPS*filefactor
if mtype == 'Samp': OAMPS[1] = filefactor
textstr = '\n'.join((
    r'$A1=%.3f$' % (OAMPS[0], ),
    r'$A2=%.3f$' % (OAMPS[1], ),
    r'$A3=%.3f$' % (OAMPS[2], ),
    r'$A4=%.3f$' % (OAMPS[3], ),
    r'$A5=%.3f$' % (OAMPS[4], ),
    r'$A6=%.3f$' % (OAMPS[5], )))
ax.text(0.85, 0.98, textstr, transform=ax.transAxes, fontsize=10, verticalalignment='top')
plt.title(ftitle)
plt.xlabel('Bx [mT]')
plt.ylabel('MOKE differential signal')
plt.tight_layout()
saving_name = os.path.splitext(filepath)[0]+'.png'
plt.savefig(saving_name)

## B field analysis

In [ ]:
# Get the folder and file
pi = 30
folder_name = r'C:\Users\migels\Miguel\PhD\MOKE\Measures\W1\Pi '+str(pi)+'\Bx'
file_name = 'noqw.h5'

# Extract the data and manipulate
filepath = os.path.join(folder_name, file_name)
data = get_averaged_loop(filepath,1000)
x = np.array(data['fields_X'])
z = np.array(data['fields_Z'])
ang = []
for i in range(len(x)):
        ang.append(abs(math.degrees(math.atan(z[i]/x[i]))))

# Here we plot the magnetic field in the three directions of the table reference frame and the angle that the field makes with x
# of the table
fig, ax1 = plt.subplots(figsize=(16, 15))
ax2 = ax1.twinx()
ax1.grid()
l1, = ax1.plot(data['t'], data['fields_X'], 'b-')
l1.set_label('Bx')
l2, = ax1.plot(data['t'], data['fields_Y'], 'g-')
l2.set_label('By')
l3, = ax1.plot(data['t'], data['fields_Z'], 'r-')
l3.set_label('Bz')
ax2.plot(data['t'],ang,color='purple')
ax1.legend()
plt.title('Study of magnetic fields (in table reference frame)')
ax1.set_xlabel('t [s]')
ax2.set_ylabel('Pi angle [°]', color='purple')
ax1.set_ylabel('B [mT]')

fig2=plt.figure(figsize=(6, 5), dpi= 200, facecolor='w', edgecolor='k')

plt.grid()
plt.plot(x,z)
plt.title('Study of magnetic fields (in table reference frame)')
plt.xlabel('Bx [mT]')
plt.ylabel('By [mT]')
plt.tight_layout()
saving_name = 'magfield.png'
plt.savefig(saving_name)

## Plot single loop

To plot a certain magnetic field direction we need the following input:
    Bx --> direction = 1
    By --> direction = 2
    Bz --> direction = 3

In [ ]:
direction = 1
loop_number = 15
direction2 = 1
loop_number2 = 2

In [ ]:
w1dif,w2dif,hall = plot_single_loop(filepath, direction, loop_number)
fig=plt.figure(figsize=(6, 3), dpi= 300, facecolor='w', edgecolor='k')
plt.rcParams.update({'font.size': 16})
plt.plot(hall,w1dif,linewidth=0.3)
plt.xlabel('B [mT]')
plt.ylabel('MOKE signal')
plt.tight_layout()
plt.show()

## Magnetic field study

In [ ]:
hall = get_magnetic_field(filepath, direction, loop_number)
hall2 = get_magnetic_field(filepath, direction2, loop_number2)
fig=plt.figure(figsize=(6, 3), dpi= 300, facecolor='w', edgecolor='k')
plt.rcParams.update({'font.size': 16})
plt.plot(hall,linewidth=0.3)
plt.plot(hall2,linewidth=0.3,color='red')
plt.xlabel('Samples')
plt.ylabel('B [mT]')
plt.tight_layout()
plt.show()